In [1]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, a456189f-28b9-4187-a1bc-a9287b33f506)

In [2]:
df = df.select("value")

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 4, Finished, Available)

In [3]:
display(df)

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 87297934-e2e1-4845-a55c-d9dbe2f3210b)

In [4]:
from pyspark.sql.functions import explode
df_explode = df.select(explode(df["value"]).alias("json_object"))

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 6, Finished, Available)

In [5]:
display(df_explode)

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 69f7a0d8-c811-4426-adf4-6e420f31f890)

In [6]:
json_list = df_explode.toJSON().collect()

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 8, Finished, Available)

In [7]:
print(json_list[29])

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 9, Finished, Available)

{"json_object":{"category":"Entertainment","datePublished":"2024-04-21T00:30:20.0000000Z","description":"In a post on Instagram on April 20, Paris Hilton shared a video of her 15-month-old son Phoenix reacting to hearing his mom's new song for the first time. The reality TV star's new track with Sia, 'Fame Won’t Love You,","mentions":[{"name":"Paris Hilton"},{"name":"New Song"},{"name":"Son"}],"name":"Paris Hilton Shares Son Phoenix’s Adorable Reaction to Hearing Her New Song: ‘Melted My Heart’","provider":[{"_type":"Organization","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=ODF.kridaNmIIz5AdgcUGG5eBQ&pid=news"}},"name":"People on MSN.com"}],"url":"https://www.msn.com/en-us/entertainment/news/paris-hilton-shares-son-phoenix-s-adorable-reaction-to-hearing-her-new-song-melted-my-heart/ar-AA1nn5mo"}}


In [8]:
import json

news_json = json.loads(json_list[29])

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 10, Finished, Available)

In [9]:
print(news_json)

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 11, Finished, Available)

{'json_object': {'category': 'Entertainment', 'datePublished': '2024-04-21T00:30:20.0000000Z', 'description': "In a post on Instagram on April 20, Paris Hilton shared a video of her 15-month-old son Phoenix reacting to hearing his mom's new song for the first time. The reality TV star's new track with Sia, 'Fame Won’t Love You,", 'mentions': [{'name': 'Paris Hilton'}, {'name': 'New Song'}, {'name': 'Son'}], 'name': 'Paris Hilton Shares Son Phoenix’s Adorable Reaction to Hearing Her New Song: ‘Melted My Heart’', 'provider': [{'_type': 'Organization', 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=ODF.kridaNmIIz5AdgcUGG5eBQ&pid=news'}}, 'name': 'People on MSN.com'}], 'url': 'https://www.msn.com/en-us/entertainment/news/paris-hilton-shares-son-phoenix-s-adorable-reaction-to-hearing-her-new-song-melted-my-heart/ar-AA1nn5mo'}}


In [10]:
print(news_json["json_object"]["name"])
print(news_json["json_object"]["description"])
print(news_json["json_object"]["category"])
print(news_json["json_object"]["url"])
print(news_json["json_object"]["provider"][0]["image"]["thumbnail"]["contentUrl"])
print(news_json["json_object"]["provider"][0]["name"])
print(news_json["json_object"]["datePublished"])

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 12, Finished, Available)

Paris Hilton Shares Son Phoenix’s Adorable Reaction to Hearing Her New Song: ‘Melted My Heart’
In a post on Instagram on April 20, Paris Hilton shared a video of her 15-month-old son Phoenix reacting to hearing his mom's new song for the first time. The reality TV star's new track with Sia, 'Fame Won’t Love You,
Entertainment
https://www.msn.com/en-us/entertainment/news/paris-hilton-shares-son-phoenix-s-adorable-reaction-to-hearing-her-new-song-melted-my-heart/ar-AA1nn5mo
https://www.bing.com/th?id=ODF.kridaNmIIz5AdgcUGG5eBQ&pid=news
People on MSN.com
2024-04-21T00:30:20.0000000Z


In [11]:
title = []
description = []
category = []
url = []
image = []
provider = []
datePublished = []

for json_str in json_list:
    try:
        article = json.loads(json_str)

        if article["json_object"].get("category") and article["json_object"].get("image",{}).get("thumbnail",{}).get("contentUrl"):       
            title.append(article["json_object"]["name"])
            description.append(article["json_object"]["description"])
            category.append(article["json_object"]["category"])
            url.append(article["json_object"]["url"])
            image.append(article["json_object"]["provider"][0]["image"]["thumbnail"]["contentUrl"])
            provider.append(article["json_object"]["provider"][0]["name"])
            datePublished.append(article["json_object"]["datePublished"])

    except Exception as e:
        print(f"error processing json object: {e}")

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 13, Finished, Available)

In [12]:
from pyspark.sql.types import StructType, StructField, StringType

data = list(zip(title,description,category,url,image,provider,datePublished))

schema = StructType([
    StructField("title",StringType(),True),
    StructField("description",StringType(),True),
    StructField("category",StringType(),True),
    StructField("url",StringType(),True),
    StructField("image",StringType(),True),
    StructField("provider",StringType(),True),
    StructField("datePublished",StringType(),True)
])

df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 14, Finished, Available)

In [13]:
display(df_cleaned)

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8b1d24dc-116c-4b83-a4aa-22566de54bf4)

In [14]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 16, Finished, Available)

In [15]:
display(df_cleaned_final.limit(5))

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, b19ea059-6a5c-4430-942f-dc77163a282e)

In [16]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_latest_news'

    df_cleaned_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table 
                    USING vw_df_cleaned_final source_view

                    ON source_view.url = target_table.url

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished

                    THEN UPDATE SET = 
                """)

StatementMeta(, d822b50a-37c1-4cbb-8df2-3fe7aa1b215b, 18, Finished, Available)